<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema2_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Coyuntura en el mercado laboral

En este notebook vamos a usar datos del mercado laboral para aprender a crear un informe de coyuntura. Como ejemplo, vamos hacer un informe de coyuntura para Sevilla.

### Fuentes estadísticas:
* INE. En la sección de [Mercado Laboral](https://www.ine.es/dyngs/INEbase/es/categoria.htm?c=Estadistica_P&cid=1254735976594), cabe destacar las siguientes estadísticas para el análisis de coyuntura:
  * [Encuesta de Población Activa (EPA)](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&menu=ultiDatos&idp=1254735976595).
  * [Encuesta trimestral de coste laboral](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736045053&menu=ultiDatos&idp=1254735976596).
* Ministerio de Trabajo:
  * [Estadística del Movimiento Laboral Registrado](https://www.mites.gob.es/es/estadisticas/mercado_trabajo/MLR/welcome.htm).
* Seguridad Social:
  * [Afiliaciones a la Seguridad Social](https://www.mites.gob.es/es/estadisticas/mercado_trabajo/MLR/welcome.htm).

*Para interpretar correctamente las variables, es fundamental conocer bien la metodología, en particular, qué miden exactamente. Así, no es comparable el concepto de "parado registrado" con el de "parado" de la EPA.*


### Diseño del informe.

Para realizar el informe, primero tenemos que decidir qué variables queremos mostrar en el mismo. Dada la disponibilidad de datos a nivel provincial, vamos a incluir las siguientes variables en nuestro informe:
1. Tasa de paro.
2. Tasa de actividad.
3. Tasa de ocupación.
4. Volumen de afiliaciones a la Seguridad Social y su tasa de variación.
5. Número de parados registrados y su tasa de variación.
  
En segundo lugar, tenemos que decidir el formato del informe. Para ello, es útil dibujar un borrador en un papel y así decidir el mejor diseño posible.  
En este caso, vamos a crear un informe breve, con la extensión de una página A4. Lo vamos a hacer enteramente en ```matplotlib```. Vamos a estructurar en informe en una figura de 6 axes (3 filas x 2 columnas). Los primeros 5 axes van a contener gráficos, mientras el último una tabla.

### Importamos librerías y datos:

In [3]:
import pandas as pd
pd.set_option('display.max_row',50)
pd.set_option('display.max_columns', 20)
import matplotlib.pyplot as plt

#### Series de la EPA
En [EPA](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&menu=ultiDatos&idp=1254735976595), hacemos clic en la siguiente secuencia: Resultados => Series desde el primer trimestre de 2002 => Resultados provinciales => Población en viviendas familiares (incluye parados) => Tasas de actividad, paro y empleo por provincia y sexo.  
Entonces, hacemos la selección deseada: las tres tasas, Ambos sexos, Total Nacional y Sevilla, todos los periodos (ordenándolos haciendo clic en a/z) y le damos a la tabla el formato deseado (con los periodos en las filas).  
Posteriormente, descargo el archivo en el formato "Excel: extensión XLSx", con el nombre de EPA_Sevilla, en cual lo subo a Google Colab y procedo a importar.

In [ ]:
epa=pd.read_excel("EPA_Sevilla.xlsx")

Tras explorar brevemente la tabla (```epa[:20]```, ```epa[-20:]```), vemos que tenemos que hacer el siguiente procesamiento:

In [12]:
# 1. Nombrar cada columna de manera apropiada
epa.rename(columns={"Unnamed: 1":"TA_esp",
                    "Unnamed: 2":"TA_sev",
                    "Unnamed: 3":"TP_esp",
                    "Unnamed: 4":"TP_sev",
                    "Unnamed: 5":"TE_esp",
                    "Unnamed: 6":"TE_sev"}, inplace=True)
# 2. Seleccionar las filas con datos (de index=8 a =93):
epa=epa.iloc[8:94] # Hay que poner 94, porque la última no se incluye

# 3. Crear una columna con el periodo, para convertirla en datetime y en el index.
epa['periodo']=epa['Resultados provinciales'].str[:4]+"Q"+epa['Resultados provinciales'].str[-1:]
epa['periodo']=pd.to_datetime(epa['periodo'])
epa.set_index('periodo', inplace=True)

# 4. Borramos columnas innecesarias
epa.drop(columns='Resultados provinciales', inplace=True)

In [ ]:
# Podemos dibujar un gráfico para ver que está todo bien.
epa.plot()

#### Parados registrados

Los datos de demandantes de empleo parados (es decir, de parados registrados) están disponibles en [este enlace](https://www.sepe.es/HomeSepe/que-es-el-sepe/estadisticas/empleo/estadisticas-nuevas.html). Como puede apreciarse, es muy tedioso descargar mes a mes, aunque podría hacerse.  
Para que aprendáis cómo podría hacerse mecánicamente, a continuación procedemos a extraer los datos necesarios directamente en Python.


In [41]:
meses={'enero':"01",'febrero':"02",'marzo':"03",'abril':"04",'mayo':"05",'junio':"06",'julio':"07",'agosto':"08",'septiembre':"09",'octubre':"10",'noviembre':"11",'diciembre':"12"}
years=["2021","2022","2023"]

preg_fecha=[]
preg_sev=[]
preg_esp=[]

for year in years:
  for mes in meses:
    link="https://www.sepe.es/SiteSepe/contenidos/que_es_el_sepe/estadisticas/datos_estadisticos/empleo/datos/"+year+"/"+mes+"_"+year+"/ESTADISTICA-DE-EMPLEO.xls"
    preg=pd.read_excel(link,sheet_name="6.2")
    fecha=year+"-"+meses[mes]
    sev=preg.loc[preg["Unnamed: 1"]=="SEVILLA","Unnamed: 2"].iloc[0]
    esp=preg.loc[preg["Unnamed: 1"]=="TOTAL ZONA GEOGRAFICA","Unnamed: 2"].iloc[0]
    preg_fecha.append(fecha)
    preg_sev.append(sev)
    preg_esp.append(esp)






HTTPError: ignored

In [45]:
preg_esp

[3964353,
 4008789,
 3949640,
 3910628,
 3781250,
 3614339,
 3416498,
 3333915,
 3257802,
 3257068,
 3182687,
 3105905,
 3123078,
 3111684,
 3108763,
 3022503,
 2922991,
 2880582,
 2883812,
 2924240,
 2941919,
 2914892,
 2881380,
 2837653,
 2908397,
 2911015,
 2862260,
 2788370,
 2739110,
 2688842,
 2677874,
 2702700,
 2722468]

In [21]:
preg

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,6.2 DEMANDAS DE EMPLEO PENDIENTES DE DEMANDANT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SEPTIEMBRE 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,TOTAL,DE 16-19 AÑOS,DE 20-24 AÑOS,DE 25-29 AÑOS,DE 30-34 AÑOS,DE 35-39 AÑOS,DE 40-44 AÑOS,DE 45-49 AÑOS,DE 50-54 AÑOS,DE 55-59 AÑOS,MAYOR DE 59 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,59,PAIS VASCO,106725,2255,6600,8404,9019,10219,11555,12809,13212,15674,16978
65,60,LA RIOJA,12679,216,645,835,976,1028,1221,1387,1571,2167,2633
66,61,CEUTA,10080,367,843,957,1114,1142,1201,1230,1191,1092,943
67,62,MELILLA,8377,274,725,905,919,880,999,1018,897,935,825
